## Sea Surface Temperature (SST) anomalies

Sea surface temperature anomalies measure the difference in temperature with respect to average conditions of the ocean. So, a positive difference means the surface ocean is warmer than normal and a negative difference means it's cooler than normal.  

In the graph below, redder colors indicate a positive difference and bluer colors indicate a negative difference. Does this make sense based on our discussion of how temperature is distributed around the Earth?

![SST anomaly](https://www.ospo.noaa.gov/data/sst/anomaly/2019/anomnight.1.3.2019.gif)

We are going to calculate sea surface temperature anomalies for the SODA dataset today.  SODA or Simple Ocean Data Assimilation combines experimental measurements of the ocean with modelled data to map out ocean conditions across the globe. The oceans are huge (almost 70% of the Earth's surface is covered by oceans) so it very difficult to get oceanographic measurements of every inch of it. Scientists instead use models based on their understanding of what influences the oceans to fill in the gaps in their measurements. They can also use models to go back in time to look at what oceans might have been like in the past.

### Pseudocode
Here are the steps we need to take:
1. import file
1. save the necessary variables
1. find the mean temperature
1. find SST anomaly

#### Step 1: Import file

```
change directory

import netcdf4 module

read in and save dataset as a variable called 'data'

look at the names of the variables
```

In [1]:
from netCDF4 import Dataset
import os
os.chdir('/Users/brownscholar/Dropbox/BridgeUP_ClimateCoders/Data')
data = Dataset('soda3.12.2_mn_ocean_reg_2017.nc')

This is a description of what each of these variables are

In [2]:
for variable in data.variables:
    print(variable,': ',data.variables[variable].long_name)


time :  time
xt_ocean :  longitude
yt_ocean :  latitude
xu_ocean :  longitude
yu_ocean :  latitude
st_ocean :  tcell zstar depth
sw_ocean :  ucell zstar depth
temp :  Potential temperature
salt :  Practical Salinity
wt :  dia-surface velocity T-points
ssh :  effective sea level (eta_t + patm/(rho0*g)) on T cells
mlt :  mixed layer depth determined by temperature criteria
mlp :  Depth of potential density mixed layer
mls :  mixed layer depth determined by salinity criteria
net_heating :  surface ocean heat flux coming through coupler and mass transfer
prho :  potential density referenced to 0 dbar
u :  i-current
v :  j-current
taux :  i-directed wind stress forcing u-velocity
tauy :  j-directed wind stress forcing v-velocity


Use the code above to print out the shape of the variables. What do the dimensions of temp i.e temperature correspond to?

In [8]:
for variable in data.variables:
    print(variable,': ',data.variables[variable].shape)


time :  (12,)
xt_ocean :  (720,)
yt_ocean :  (330,)
xu_ocean :  (720,)
yu_ocean :  (330,)
st_ocean :  (50,)
sw_ocean :  (50,)
temp :  (12, 50, 330, 720)
salt :  (12, 50, 330, 720)
wt :  (12, 50, 330, 720)
ssh :  (12, 330, 720)
mlt :  (12, 330, 720)
mlp :  (12, 330, 720)
mls :  (12, 330, 720)
net_heating :  (12, 330, 720)
prho :  (12, 50, 330, 720)
u :  (12, 50, 330, 720)
v :  (12, 50, 330, 720)
taux :  (12, 330, 720)
tauy :  (12, 330, 720)


#### Step 2: Save the required variables

```
save the variables temperature depends on i.e time, st_ocean, yt_ocean, xt_ocean

save temperature i.e temp
```

In [4]:
# Save latitude as variable
lat = data.variables['yt_ocean'][:]
# Save longitude as variable
lon =  data.variables['xt_ocean'][:]
#Save time as variable
time = data.variables['time'][:]
#Save depth as variable
depth = data.variables['st_ocean'][:]
# Save temperature as variable
temp = data.variables['temp'][:]

In [5]:
# Save temperature as variable

Print out the minimum and maximum of each variable. Do these values make sense?

In [6]:
import numpy as np

#You can use the min() and max() *method* to do this calculation.
#lon = np.array([1,2,3,4,5])
#print('Example: ',lon.min(),'\n')

print(lon.min())
print(lon.max())

0.25
359.75


Notice that time is measured in a different format as days from Jan 01, 1980. Sound familiar? We'll learn how to convert to a datetime format using the function in another session.

Draw out what the temp variable might look like.

#### Step 3: Find mean temperature 

*Hint*: Use https://docs.scipy.org/doc/numpy/reference/generated/numpy.ma.mean.html

First, we need to decide what depth we would consider to be surface ocean? Call this variable z.

In [10]:
z = temp[0]

Then, let's do this step by step. 

1. Only obtain data for the first month. You should get a grid with a shape of (50, 330, 720) 

In [11]:
print(z)

[[[-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  ...
  [-1.7133582830429077 -1.7133338451385498 -1.7132887840270996 ...
   -1.713502049446106 -1.7134593725204468 -1.7134010791778564]
  [-1.7146706581115723 -1.7147420644760132 -1.7147945165634155 ...
   -1.714470624923706 -1.714550256729126 -1.7146133184432983]
  [-1.719269871711731 -1.7193185091018677 -1.7193660736083984 ...
   -1.7191441059112549 -1.7191851139068604 -1.719224214553833]]

 [[-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  [-- -- -- ... -- -- --]
  ...
  [-1.7133210897445679 -1.7132916450500488 -1.7133280038833618 ...
   -1.7135671377182007 -1.7134387493133545 -1.7133781909942627]
  [-1.7134846448898315 -1.713546872138977 -1.7135862112045288 ...
   -1.7136391401290894 -1.7135635614395142 -1.7134833335876465]
  [-1.7201088666915894 -1.720162034034729 -1.7202140092849731 ...
   -1.7199562788009644 -1.7200077772140503 -1.7200566530227661]]

 [[-- -- -- ... -- -- --]
  [-- -- -- ... -- -- -

1. Identify the index for the first z m of the ocean
1. Save temperature for the top z m

In [13]:
depth <20

masked_array(data=[ True,  True, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False],
             mask=False,
       fill_value=True)

In [15]:
temp_jan = temp[0,depth<20,:,:]

Average over all z m. You should get a grid that is 330 by 720.

In [16]:
temp_jan_mean = temp_jan.mean(axis = 0)
temp_jan_mean.shape

(330, 720)

Now, find the mean for the entire grid.

In [17]:
temp_jan_mean.mean()

14.015762457458468

#### But....

You can do this step all at once by just applying the mean function to your array of the first month down to a depth of z m. I just wanted to make sure you got the steps its taking.

Now find the mean for all 12 months to a depth of z m.

In [19]:
temp_mean = temp[0,depth<20,:,:].mean()

#### Step 3: Find SST anomaly

Remember:

SST anomaly = SST - average SST

In [20]:
temp[:, depth<20,:,:] - temp_mean

masked_array(
  data=[[[[--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          ...,
          [-15.729121208190918, -15.729097366333008,
           -15.72905158996582, ..., -15.729265213012695,
           -15.729222297668457, -15.729164123535156],
          [-15.73043441772461, -15.730504989624023,
           -15.730557441711426, ..., -15.730234146118164,
           -15.730313301086426, -15.730376243591309],
          [-15.73503303527832, -15.735081672668457,
           -15.735129356384277, ..., -15.734907150268555,
           -15.73494815826416, -15.734987258911133]],

         [[--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          ...,
          [-15.729084014892578, -15.729055404663086,
           -15.72909164428711, ..., -15.729330062866211,
           -15.729202270507812, -15.729141235351562],
          [-15.729248046875, -15.729310035705566, -15

## Challenge

Repeat these steps for sea surface salinity data ('salt')